In [8]:

from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3

from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4
from KNN.SuperHybridRecommender import SuperHybridRecommender

from KNN.UserKNNCBFRecommender import UserKNNCBFRecommender

from KNN.Hybrid_List import Hybrid_List
from evaluation_function import evaluate_algorithm


#from GraphBased.RP3betaRecommender import RP3betaRecommender
from GraphBased.P3alphaRecommender import P3alphaRecommender

In [9]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')


#Load playlistsIDS
train_df= pd.read_csv('all/train.csv',low_memory = False)
all_userID=train_df.playlist_id.unique()

In [ ]:
CFrecommender = ItemKNNCFRecommender(URM_train)
CFrecommender.fit()

SLIMrecommender = SLIM_BPR_Cython(URM_train)
SLIMrecommender.fit()

CBFrecommender = ItemKNNCBFRecommender(ICM_all,URM_train)
CBFrecommender.fit()

SLIMErecommender = SLIMElasticNetRecommender(URM_train)
SLIMErecommender.fit()

UCFRecommender = UserKNNCFRecommender(URM_train)
UCFRecommender.fit()

SVDrecommender = PureSVDRecommender (URM_train)
SVDrecommender.fit()

p3Arecommender=P3alphaRecommender(URM_train)
p3Arecommender.fit()

UserCBFrecommender = UserKNNCBFRecommender(UCM_all,URM_train) 
UserCBFrecommender.fit()

SVDrecommender = PureSVDRecommender (URM_train)
SVDrecommender.fit()

In [ ]:
#evaluate a recommender with MAP10
evaluate_algorithm(URM_test,CFrecommender)

In [27]:
#create an Hybrid Recommender and evaluate it
#specify the recommenders to hrybrid during the creation
Hybird_recommender= ItemKNNSimilarityHybridRecommender (URM_train,CFrecommender,CBFrecommender)
Hybird_recommender.fit()
evaluate_algorithm(URM_test,Hybird_recommender)

In [ ]:
#some more complicated Hybrids
SuperScores=ItemKNNScoresHybridRecommender(URM_train,SLIMrecommender,SLIMErecommender)
SuperScores.fit(alpha=0.5)

H4Scores1=ItemKNNScoresHybridRecommender4(URM_train,SuperScores,CFrecommender,UCFRecommender, CBFrecommender) 
H4Scores1.fit(alpha=0.5, beta=0.5, gamma=0.2, delta=0.2)
result = H4Scores1.evaluateRecommendations(URM_test)
print("Recommender MAP is= {}".format(result["MAP"]))